# Build Validation Table for NPMRDS
For each of the 9 SACSIM time periods, provide:
* Average speed
* Number of epochs
* Standard error of the speed (standard deviation / mean)

In [2]:
import os
from time import perf_counter as perf

import pandas as pd
import urllib
import sqlalchemy as sqla # needed to run pandas df.to_sql() function
    
# extract SQL Server query results into a pandas dataframe   
def sqlqry_to_df(query_str, dbname, servername='SQL-SVR', trustedconn='yes'):     

    conn_str = "DRIVER={ODBC Driver 17 for SQL Server};" \
        f"SERVER={servername};" \
        f"DATABASE={dbname};" \
        f"Trusted_Connection={trustedconn}"
        
    conn_str = urllib.parse.quote_plus(conn_str)
    engine = sqla.create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")
       
    start_time = perf()

    # create SQL table from the dataframe
    print("Executing query. Results loading into dataframe...")
    df = pd.read_sql_query(sql=query_str, con=engine)
    rowcnt = df.shape[0]
    
    et_mins = round((perf() - start_time) / 60, 2)
    print(f"Successfully executed query in {et_mins} minutes. {rowcnt} rows loaded into dataframe.")
    
    return df

def get_prd_df(qry_file, db_name, speed_tbl, tmc_tbl, prd_tag, prd_hrs):
    
    with open(qry_file, 'r') as f:
        q_str = f.read()
        
    q_str_formatted = q_str.format(speed_tbl, prd_hrs, prd_tag, tmc_tbl)
    
    prd_df = sqlqry_to_df(q_str_formatted, db_name)
    
    return prd_df

print("loaded all modules. ready to proceed.")
    

loaded all modules. ready to proceed.


In [3]:
query_file = 'cong_speed_model_prds.sql'

db = 'NPMRDS'
speed_data_year = 2017
TMC_yr = 2020
tbl_speeds = 'npmrds_2017_alltmc_paxtruck_comb'
tbl_tmcs = 'npmrds_2020_alltmc_txt'
join_key = 'tmc'

prds = {'h07':[7], 'h08':[8], 'h09':[9], 'md5':[10, 11, 12, 13, 14], 
        'h15':[15], 'h16':[16], 'h17':[17], 'ev2':[18, 19], 
        'n11': [20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6]}

df_master = pd.DataFrame()

for i, prdtag in enumerate(prds.keys()):
    print(f"loading in data for {prdtag}...")
    prd_hours = ','.join(str(i) for i in prds[prdtag])
    if i == 0:
        df_master = get_prd_df(qry_file=query_file, db_name=db,
                               speed_tbl=tbl_speeds, tmc_tbl=tbl_tmcs, 
                               prd_tag=prdtag, prd_hrs=prd_hours)
    else:
        df_p = get_prd_df(qry_file=query_file, db_name=db,
                               speed_tbl=tbl_speeds, tmc_tbl=tbl_tmcs, 
                               prd_tag=prdtag, prd_hrs=prd_hours)
        
        df_master = df_master.merge(df_p, on=join_key)
        
        
print("successfully created dataframe for speeds in all sacsim time periods.")
        

    


loading in data for h07...
Executing query. Results loading into dataframe...
Successfully executed query in 0.55 minutes. 7495 rows loaded into dataframe.
loading in data for h08...
Executing query. Results loading into dataframe...
Successfully executed query in 0.41 minutes. 7495 rows loaded into dataframe.
loading in data for h09...
Executing query. Results loading into dataframe...
Successfully executed query in 0.35 minutes. 7495 rows loaded into dataframe.
loading in data for md5...
Executing query. Results loading into dataframe...
Successfully executed query in 0.45 minutes. 7495 rows loaded into dataframe.
loading in data for h15...
Executing query. Results loading into dataframe...
Successfully executed query in 0.27 minutes. 7495 rows loaded into dataframe.
loading in data for h16...
Executing query. Results loading into dataframe...
Successfully executed query in 0.25 minutes. 7495 rows loaded into dataframe.
loading in data for h17...
Executing query. Results loading into

In [4]:
# join to link feature class and export to GIS feature class
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import datetime as dt

input_link_fc = r'Q:\SACSIM23\Network\SM23GIS\MN_link_forConflation_YZ\ConflationResults.gdb\conflation_INRIX_20211207'
links_jnky = 'Tmc'
link_fc_cols = ['OBJECTID', 'A', 'B', 'A_B', 'CAPC20', 'NAME', links_jnky,
       'RoadName', 'FwyTag', 'RoadNumber', 'Type', 'Mis_cf_check', 'SHAPE']


output_fgdb = r'Q:\SACSIM23\Network\SM23GIS\SM23Testing.gdb'

#-----------RUN script-----------
sufx = str(dt.datetime.now().strftime('%Y%m%d_%H%M'))

output_fc_name = f'InrixPrdSpd_{speed_data_year}on{TMC_yr}TMC{sufx}'


sedf_links = pd.DataFrame.spatial.from_featureclass(input_link_fc)
sedf_links = sedf_links[link_fc_cols]
sedf_links = sedf_links.merge(df_master, how='left', left_on=links_jnky, right_on=join_key)
sedf_links.spatial.to_featureclass(os.path.join(output_fgdb, output_fc_name))

'Q:\\SACSIM23\\Network\\SM23GIS\\SM23Testing.gdb\\InrixPrdSpd_2017on2020TMC20220119_1050'